## Start by setting week number
#### Wait for ESPN data update until at least Wednesday morning, but run latest by Thursday night

https://www.espn.com/nfl/stats/team/_/stat/total/season/2021/seasontype/2

In [1]:
SET_WEEK = 18
week_name = 'week_' + str(SET_WEEK)
print(week_name)

week_18


In [2]:
from scrapers.scrape_2021 import get_2021_data
import pandas as pd
import numpy as np
import pickle

import warnings
warnings.filterwarnings('ignore')

### Models

In [3]:
rf = '../modeling/rf.pkl'
gbm = '../modeling/gbm.pkl'
lgb = '../modeling/lgb.pkl'
catboost = '../modeling/catboost.pkl'

with open(rf, 'rb') as file:
    RF = pickle.load(file)
    
with open(gbm, 'rb') as file:
    GBM = pickle.load(file)
    
with open(lgb, 'rb') as file:
    LGB = pickle.load(file)
    
with open(catboost, 'rb') as file:
    CATBOOST= pickle.load(file)

### 2021 data

In [4]:
%%time

X = get_2021_data()

# from boruta elimination method:
keep_cols = ['offense_points_per_game', 'offense_passing_AVG', 'offense_passing_YDS/G', 'offense_passing_RTG', 
             'offense_receiving_AVG', 'offense_rushing_YDS/G', 'defense_points_per_game', 'defense_passing_AVG', 
             'defense_receiving_AVG', 'defense_rushing_YDS/G', 'defense_passing_SYL_per_game', 
             'offense_downs_Third Downs_ATT_per_game', 'defense_downs_Third Downs_ATT_per_game', 
             'defense_downs_Fourth Downs_ATT_per_game', 'defense_pass_to_rush_ratio', 
             'defense_downs_First Downs_penalty_ratio', 'offense_passing_TD_per_game', 'offense_pass_TD_to_INT']

X = X[['Team', 'games_played', 'season'] + keep_cols].copy()

X.head(2)

Wall time: 21.7 s


,Team,games_played,season,offense_points_per_game,offense_passing_AVG,offense_passing_YDS/G,offense_passing_RTG,offense_receiving_AVG,offense_rushing_YDS/G,defense_points_per_game,...,defense_receiving_AVG,defense_rushing_YDS/G,defense_passing_SYL_per_game,offense_downs_Third Downs_ATT_per_game,defense_downs_Third Downs_ATT_per_game,defense_downs_Fourth Downs_ATT_per_game,defense_pass_to_rush_ratio,defense_downs_First Downs_penalty_ratio,offense_passing_TD_per_game,offense_pass_TD_to_INT
0,Dallas Cowboys,17,2021,31.2,7.7,282.4,104.8,11.2,124.6,21.1,...,11.9,112.8,17.470588,13.411765,13.117647,1.529412,1.426573,0.113150,2.352941,3.636364
1,Tampa Bay Buccaneers,17,2021,30.1,7.4,307.6,101.6,10.9,98.4,20.8,...,9.8,92.5,16.764706,13.117647,13.294118,1.647059,1.857923,0.104651,2.529412,3.583333


In [5]:
X.fillna(0, inplace = True)
X.replace([np.inf, -np.inf], 0, inplace=True)

In [6]:
X['games_played'].unique().tolist()

[17]

In [7]:
# X[X['games_played'] == 8]

In [8]:
predictions = X[['Team', 'season']].copy()
predictions['week_marker'] = week_name

predictions['pred_rf'] = RF.predict_proba(X.drop(['Team', 'games_played', 'season'], 1))[:,1]
predictions['pred_gbm'] = GBM.predict_proba(X.drop(['Team', 'games_played', 'season'], 1))[:,1]
predictions['pred_lgb'] = LGB.predict_proba(X.drop(['Team', 'games_played', 'season'], 1))[:,1]
predictions['pred_catboost'] = CATBOOST.predict_proba(X.drop(['Team', 'games_played', 'season'], 1))[:,1]
predictions['pred_mean'] = (predictions['pred_rf'] + predictions['pred_gbm'] + predictions['pred_lgb'] + predictions['pred_catboost']) / 4

predictions.sort_values('pred_mean', ascending = False, inplace = True)
predictions.head(3)

,Team,season,week_marker,pred_rf,pred_gbm,pred_lgb,pred_catboost,pred_mean
0,Dallas Cowboys,2021,week_18,0.248449,0.238467,0.219779,0.243677,0.237593
1,Tampa Bay Buccaneers,2021,week_18,0.240275,0.192649,0.151929,0.251427,0.209070
4,Buffalo Bills,2021,week_18,0.219198,0.140607,0.185484,0.206680,0.187992


In [9]:
predictions.to_csv('weekly_runs/' + week_name + '.csv', index = False)

### Check weekly run

In [10]:
predictions

,Team,season,week_marker,pred_rf,pred_gbm,pred_lgb,pred_catboost,pred_mean
0,Dallas Cowboys,2021,week_18,0.248449,0.238467,0.219779,0.243677,0.237593
1,Tampa Bay Buccaneers,2021,week_18,0.240275,0.192649,0.151929,0.251427,0.209070
4,Buffalo Bills,2021,week_18,0.219198,0.140607,0.185484,0.206680,0.187992
8,Los Angeles Rams,2021,week_18,0.196796,0.129612,0.173841,0.229678,0.182482
6,San Francisco 49ers,2021,week_18,0.168037,0.108617,0.179985,0.233008,0.172412
2,Kansas City Chiefs,2021,week_18,0.190713,0.095121,0.104417,0.201418,0.147917
14,New England Patriots,2021,week_18,0.120356,0.097770,0.089911,0.179285,0.121831
12,Cincinnati Bengals,2021,week_18,0.124189,0.071001,0.095197,0.186677,0.119266
3,Los Angeles Chargers,2021,week_18,0.144671,0.070768,0.082983,0.167955,0.116594
9,Green Bay Packers,2021,week_18,0.092273,0.042917,0.067423,0.201111,0.100931
